## Chat sobre documentos PDF : RAG con LangChain, Ollama, y FAISS Vector Store

In [ ]:
# pip install -U langchain-community faiss-cpu langchain-huggingface pymupdf tiktoken langchain-ollama python-dotenv

In [1]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

### Document Loader

In [10]:
from langchain_community.document_loaders import PyMuPDFLoader

# loader = PyMuPDFLoader("./dgi-dataset/*.pdf")
# docs = loader.load()
# load multiple pdfs
# poner directorio donde se encuentran los pdfs
pdf_directory = ".\dgi-dataset"
documents = []

for filename in os.listdir(pdf_directory):
    if filename.endswith(".pdf"):
        loader = PyMuPDFLoader(os.path.join(pdf_directory, filename))
        documents.extend(loader.load())

In [11]:
documents[0]

Document(metadata={'source': '.\\dgi-dataset\\798+12+web+06+024 (2).pdf', 'file_path': '.\\dgi-dataset\\798+12+web+06+024 (2).pdf', 'page': 0, 'total_pages': 25, 'format': 'PDF 1.7', 'title': 'Microsoft Word - 27F1BEE0.doc', 'author': 'Ileana Olivera', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'Microsoft: Print To PDF', 'creationDate': "D:20240809093858-03'00'", 'modDate': "D:20240809095909-03'00'", 'trapped': ''}, page_content=' \nActualización Junio 2024 \n \nRESOLUCIÓN D.G.I. Nº 798/2012 \n \n \nDOCUMENTACIÓN FISCAL ELECTRÓNICA –SE ESTABLECEN LAS CONDICIONES QUE REGULEN SU \nRÉGIMEN. \n \n \nMontevideo, 8 de mayo de 2012 \n \nVisto: la Ley Nº 18.600 de 21 de setiembre de 2009 y el Decreto Nº 36/012 de 8 de febrero de \n2012. \n \nResultando: I) Que la citada norma legal reconoce la validez y la eficacia jurídica de los \ndocumentos electrónicos y la firma electrónica; \n \nII) Que el mencionado decreto comete a la Dirección General Impositiva el dictado de normas \nc

In [12]:
import os

pdfs = []
for root, dirs, files in os.walk('dgi-dataset'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

In [13]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)

In [14]:
len(docs)

305

### Document Chunking

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)

In [16]:
len(docs), len(chunks)

(305, 830)

In [17]:
len(docs[0].page_content), len(chunks[0].page_content)

(2451, 922)

In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

len(encoding.encode(docs[0].page_content)), len(encoding.encode(chunks[0].page_content))

(632, 245)

### Document Vector Embedding

In [19]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [20]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

single_vector = embeddings.embed_query("this is some text data")


In [21]:
len(single_vector)

768

In [22]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d

(0, 768)

In [23]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [24]:
len(chunks)

830

In [25]:
ids = vector_store.add_documents(documents=chunks)

In [26]:
vector_store.index_to_docstore_id
len(ids)

830

In [27]:
# # store vector database
db_name = "db_name"
vector_store.save_local(db_name)

# # # load vector database
new_vector_store = FAISS.load_local(db_name, embeddings=embeddings, allow_dangerous_deserialization=True)
len(new_vector_store.index_to_docstore_id)

830

### Retreival

In [28]:
question = "¿Cuándo debo hacer una nota de crédito?"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")

son: 
 e-Factura. 
 
 Nota de crédito de e- Factura. 
 
Para operaciones entre contribuyentes. 
 Nota de débito de e- Factura. 
 
 
(B2B) 
 
 e-Ticket. 
 Nota de crédito de e-Ticket. 
 
Sólo para Consumo Final. 
 Nota de débito de e-Ticket.  
 
 
(B2C) 
 
 e- Factura de Exportación 
 Nota de crédito de e- Factura de Exportación     Sólo para exportaciones. 
 Nota de débito de e-Factura de Exportación



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia



Cuenta Ajena, e-Boleta de entrada, Nota de Crédito de e-Boleta de entrada y Nota de 
Débito de e-Boleta de entrada>  
 
de conting

In [29]:
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs = {'k': 3, 
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})

In [30]:
docs = retriever.invoke(question)

for doc in docs:
     print(doc.page_content)
     print("\n\n")


son: 
 e-Factura. 
 
 Nota de crédito de e- Factura. 
 
Para operaciones entre contribuyentes. 
 Nota de débito de e- Factura. 
 
 
(B2B) 
 
 e-Ticket. 
 Nota de crédito de e-Ticket. 
 
Sólo para Consumo Final. 
 Nota de débito de e-Ticket.  
 
 
(B2C) 
 
 e- Factura de Exportación 
 Nota de crédito de e- Factura de Exportación     Sólo para exportaciones. 
 Nota de débito de e-Factura de Exportación



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia



252 
Nota de Crédito de e-Boleta de entrada 
153 
Nota de Débito de e-Boleta de entrada 
253 
Nota de Débito de e-Boleta de entrada 
181 
e-Remito 
281 
e-Remito Contingencia 
182 
e-Resguardo 
282 
e-Resguardo Contingencia





In [31]:


question = "¿Cuándo debo hacer una nota de crédito?"
docs = retriever.invoke(question)


### RAG con LLAMA 3.2 sobre OLLAMA

In [32]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_ollama import ChatOllama

from langchain_openai import ChatOpenAI

In [33]:
model = ChatOllama(model="llama3.2:1b", base_url="http://localhost:11434")

#model = ChatOpenAI(model="gpt-4o")

#model.invoke("hi")

In [34]:
prompt = hub.pull("rlm/rag-prompt")

In [35]:
prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know. 
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [36]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# print(format_docs(docs))

In [37]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [39]:
question = "¿Cuándo debo hacer una nota de crédito?"
output = rag_chain.invoke(question)
print(output)


Aquí te presento las respuestas en forma de puntos:

• Para operaciones entre contribuyentes, para operaciones entre contribuyentes y solo para consumo final (B2B).
• Para exportaciones, sólo para exportaciones (B2C), e-Factura, Nota de crédito de e- Factura, e-Boleta de entrada.


In [40]:
question = "¿Qué es un RDC?"
output = rag_chain.invoke(question)
print(output)

* La opción "ANO2025 Programar RDC" permite generar un Reporte Diario Consolidado (RDC) con información de comprobantes fiscales electrónicos emitidos por cada tipo de CFE/CFC.
 * Los filtros de la búsqueda permiten acceder a los reportes generados para las empresas de la plataforma, incluyendo:
 + RUC de la empresa a la que se le genera el RDC
 + Fecha al que corresponde el RDC
 + Estado del RDC: aceptado, almacenado o rechazado
* La opción "ANO2025 Programar RDC" también permite configurar los valores de parámetros de tipo "global" o "global y unidad ejecutora", lo que permite a los administradores personalizar los resultados.
 * Los filtros de la búsqueda permiten seleccionar una categoría específica para el parámetro del RDC.


In [41]:
question = "¿Qué es un CFE?"
output = rag_chain.invoke(question)
print(output)

Aquí te presento las respuestas en forma de puntos:

* ¿Qué es un CFE? - Un Comprobante Fiscal de Contingencia es un tipo de comprobante papel asignado por la DGI para situaciones de contingencia.
* ¿Cuál es el conjunto mínimo obligatorio de comprobantes fiscales electrónicos con los que se puede ingresar en el sistema? - El conjunto mínimo obligatorio incluye e-Factura y e-Ticket, junto con sus notas de crédito y débito para correcciones y ajustes al documento original.
* ¿Qué direcciones de correo electrónico se constituyen al postularse en el formulario "Solicitud de Ingreso al Sistema de CFE"? - No se proporciona información específica sobre las direcciones de correo electrónico necesarias para la solicitud.


### Gradio Interface

In [89]:
import gradio as gr 

In [90]:
def chatbot(question):
    															  
    docs = retriever.invoke(question)

    rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser())

    output = rag_chain.invoke(question)

    return output

    # streams option
    # result = ""
    # for chunk in stream:
    #     result += chunk.choices[0].delta.content or ""
    #     yield result


In [ ]:
view = gr.Interface(
    fn=chatbot,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=20)],
    flagging_mode="never"
)

gr.themes.Ocean() 

view.launch(share=True)